In [2]:
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser,# A synchronous browser is available, though it isn't compatible with jupyter.
)

In [3]:
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [4]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(name='click_element', description='Click on an element with the given CSS selector', args_schema=<class 'langchain.tools.playwright.click.ClickToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, sync_browser=None, async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/yifan/.cache/ms-playwright/chromium-1060/chrome-linux/chrome> version=113.0.5672.53>),
 NavigateTool(name='navigate_browser', description='Navigate a browser to the specified URL', args_schema=<class 'langchain.tools.playwright.navigate.NavigateToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, sync_browser=None, async_browser=<Browser type=<BrowserType name=chromium executable_path=/home/yifan/.cache/ms-playwright/chromium-1060/chrome-linux/chrome> version=113.0.5672.53>),
 NavigateBackTool(name='previous_webpage', description='Navigate back to the previous page in the browser history', args_schema=<class 'pydantic.m

In [5]:
tools_by_name = {tool.name: tool for tool in tools}
navigate_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]

In [6]:
await navigate_tool.arun({"url": "https://web.archive.org/web/20230428131116/https://www.cnn.com/world"})

'Navigating to https://web.archive.org/web/20230428131116/https://www.cnn.com/world returned status code 200'

In [7]:
# The browser is shared across tools, so the agent can interact in a stateful manner
await get_elements_tool.arun({"selector": ".container__headline", "attributes": ["innerText"]})

'[{"innerText": "These Ukrainian veterinarians are risking their lives to care for dogs and cats in the war zone"}, {"innerText": "Life in the ocean’s ‘twilight zone’ could disappear due to the climate crisis"}, {"innerText": "Clashes renew in West Darfur as food and water shortages worsen in Sudan violence"}, {"innerText": "Thai policeman’s wife investigated over alleged murder and a dozen other poison cases"}, {"innerText": "American teacher escaped Sudan on French evacuation plane, with no help offered back home"}, {"innerText": "Dubai’s emerging hip-hop scene is finding its voice"}, {"innerText": "How an underwater film inspired a marine protected area off Kenya’s coast"}, {"innerText": "The Iranian drones deployed by Russia in Ukraine are powered by stolen Western technology, research reveals"}, {"innerText": "India says border violations erode ‘entire basis’ of ties with China"}, {"innerText": "Australian police sift through 3,000 tons of trash for missing woman’s remains"}, {"in

In [8]:
# If the agent wants to remember the current webpage, it can use the `current_webpage` tool
await tools_by_name['current_webpage'].arun({})

'https://web.archive.org/web/20230428133211/https://cnn.com/world'

# Use within an Agent

In [9]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatAnthropic, ChatOpenAI

llm = ChatOpenAI(temperature=0) # or any other LLM, e.g., ChatOpenAI(), OpenAI()

agent_chain = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [10]:
result = await agent_chain.arun("What are the headers on langchain.com?")
print(result)



> Entering new AgentExecutor chain...
Thought: I can use the get_elements tool to retrieve the headers on the website.

Action:
```
{
  "action": "get_elements",
  "action_input": {
    "selector": "h1, h2, h3, h4, h5, h6",
    "attributes": ["textContent"]
  }
}
```


Observation: []
Thought:It seems like the previous action did not return any results. I can try using the extract_text tool to retrieve all the text on the webpage and manually search for the headers.

Action:
```
{
  "action": "extract_text",
  "action_input": {}
}
```


Observation: World news - breaking news, video, headlines and opinion | CNN The Wayback Machine - https://web.archive.org/web/20230428133211/https://www.cnn.com/world CNN values your feedback 1. How relevant is this ad to you? 2. Did you encounter any technical issues? No Video player was slow to load content Video content never loaded Ad froze or did not finish loading Video content did not start after ad Audio on ad was too loud Other issues Ad neve